In [0]:
#
# Executing this cell you will install OpenMC and the nuclear
# data libraries in this instance of the Google Colaboratory virtual machine.
# The process takes about 2 minutes.
#

def install_openmc():
  #
  # Clone source code from Github, make and install
  #
  
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  
  %cd -q /content
  print("Obtaining OpenMC source code from Github...")
  !git  --no-pager clone https://github.com/mit-crpg/openmc.git &> /dev/null
  %cd -q openmc
  !git --no-pager checkout master &> /dev/null
  !mkdir build
  %cd -q build
  print("Running cmake...")
  !cmake ../ -DPYTHON_EXECUTABLE=/usr/bin/python3 -DCMAKE_INSTALL_PREFIX=/usr/local &> /dev/null
  print("Running make...")
  !make -j &> /dev/null
  print("Running make install...")
  !make install &> /dev/null
  import sys
  sys.path.append('/usr/local/lib/python3.6/dist-packages/openmc-0.10.0-py3.6-linux-x86_64.egg')
  sys.path.append('/usr/local/lib/python3.6/site-packages/')
  %cd -q /content

def install_data_from_onedrive():
  #
  # Download preprocessed HDF5 files from OneDrive (faster).
  #
  import os
  
  if not os.path.isdir('/content'):
    print("Esta función instala los datos nucleares de OpenMC en una instancia de Google Colaboratory.")
    print("Para instalar localmente siga las instrucciones de la documentacion:")
    print("http://docs.openmc.org/en/stable/quickinstall.html")
    return
  %cd -q /content
  print("Obtaining HDF5 files from OneDrive...")
  !wget -O nndc_hdf5.tar.gz "https://onedrive.live.com/download?cid=22422A8EEA2A85B9&resid=22422A8EEA2A85B9%21105&authkey=AHL6xwYFXDwEzkk" &> /dev/null
  print("Uncompressing...")
  !tar xzvf nndc_hdf5.tar.gz &> /dev/null
    
from time import time
t1 = time()
install_openmc()
install_data_from_onedrive()
t2 = time()
print("Installed OpenMC in {:.2f} minutes".format((t2-t1)/60.0))


# Ejemplos de definición de superficies y regiones en OpenMC

Comenzamos importando el módulo `openmc`, contiene el API (applicationg programming interface) para generar el input de OpenMC.

In [0]:
import openmc

La definición de las distintas superficies se realiza con funciones del paquete `openmc`. Por ejemplo, para generar un plano perpendicular al eje $x$ hacemos:

In [0]:
plano = openmc.XPlane()

In [0]:
print(type(plano))

El objeto de clase `XPlane` posee una serie de atributos y métodos que permiten modificarlo. Por ejemplo, para modificar el punto en que corta el eje $x$ hacemos:

In [0]:
plano.x0 = 10.0
plano.name = "Plano 1"

Esto es equivalente a crear el objeto dándole los valores como parámetros a la función `XPlane()`:

In [0]:
plano2 = openmc.XPlane(x0=10.0, name="Plano 2")

Una superficie perpendicular al eje $x$ está definida por la ecuación $x-x_0=0$, por lo que es necesario definir un único parámetro ($x_0$) para definir el plano. De forma similar para planos perpendiculares a los ejes $y$ y $z$:

In [0]:
plano3 = openmc.YPlane(y0=5)
plano4 = openmc.ZPlane(z0=3)

Para definir una esfera es necesario indicar la posición $x_0, y_0, z_0$ del centro, y su radio ya que está definida por $(x-x_0)^2+(y-y_0)^2+(z-z_0)^2-R^2=0$:

In [0]:
esfera = openmc.Sphere(x0=10, R=100)

(los valores $y_0$ y $z_0$ que no fueron especificados toman su valor por defecto, 0)

Se pueden definir cilindros paralelos a los ejes $x$, $y$, $z$ respectivamente utilizando las funciones `XCylinder()`, `YCylinder()` y `ZCylinder()`:

In [0]:
# Un cilindro paralelo al eje x de 10 cm de radio, que pasa por y=2 cm, z=1 cm
cil1 = openmc.XCylinder(y0=2, z0=1, R=10) 

# Un cilindro centrado en el eje y de radio 5 cm
cil2 = openmc.YCylinder(R=5) 

# Un cilindro centrado en el eje z de radio 1 cm
cil3 = openmc.ZCylinder() 

Además existen las funciones:
* `Plane()`: define un plano con orientación arbitraria.
* `XCone()`: define un cono con eje paralelo al eje $x$.
* `YCone()`: define un cono con eje paralelo al eje $y$.
* `ZCone()`: define un cono con eje paralelo al eje $z$.
* `Quadric()`: Define una superficie cuadrática arbitraria que responde a la ecuación $Ax^2+By^2+Cz^2+Dxy+Eyz+Fxz+Gx+Hy+Jz+K=0$


Cada superficie que generamos tiene un identificador. Si no lo indicamos explícitamente al invocar la función que genera las superficies estos identificadores se asignan en forma secuencial:

In [0]:
print(plano.id)
print(cil3.id)

Estos identificadores parecen en principio redundantes, ya que cada superficie tiene su objeto y potencialmente un nombre que la identifica. Pero, debido a que en otros códigos Monte Carlo suelen identificar las superficies con números enteros, es conveniente asignar los valores en la incialización de las superficies para facilitar la comparación:

In [0]:
otro_plano = openmc.XPlane(x0=10, surface_id=999, name="Otro Plano")
print(otro_plano.id)

## Regiones

Una vez que se creó una superficie, los operadores `+` y `-` permiten definir semiespacios:

In [0]:
adelante = +plano2
atras = -plano2
print(type(adelante))

Estos semiespacios pueden combinarse para formar regiones utilizando los operadores `&` (intersección), `|` (unión) y `~` (complemento). Por ejemplo, si quisieramos definir la región dentro de una esfera y arriba de su plano medio podemos hacer:

In [0]:
# Defino dos superficies:
esfera = openmc.Sphere(R=10)
plano_medio = openmc.ZPlane(z0=0)

# Defino algunas semiespacios en base a estas superficies
adentro_de_la_esfera = -esfera
arriba_del_plano_medio = +plano_medio

# Combino estas regiones para generar nuevas regiones
hemisferio_norte = adentro_de_la_esfera & arriba_del_plano_medio
print(type(hemisferio_norte))

La región dentro de la esfera pero abajo del plano medio es:

In [0]:
hemisferio_sur = adentro_de_la_esfera & ~hemisferio_norte

A veces para hacer estas construcciones es útil recordar las reglas de De Morgan:
* `~(A | B) = ~A & ~B`
* `~(A & B) = ~A | ~B`

Utilizando esto otra forma de definir esta región es:

In [0]:
hemisferio_sur = adentro_de_la_esfera & ~(adentro_de_la_esfera & arriba_del_plano_medio)

In [0]:
hemisferio_sur = adentro_de_la_esfera & (~adentro_de_la_esfera | ~arriba_del_plano_medio)

In [0]:
hemisferio_sur = -esfera & (~ -esfera | ~ +plano_medio)

In [0]:
hemisferio_sur = -esfera & ( +esfera | -plano_medio)

In [0]:
hemisferio_sur = (-esfera &  +esfera) | (-esfera & -plano_medio)

In [0]:
hemisferio_sur = (-esfera & -plano_medio)

Es conveniente definir las regiones de la forma más sencilla posible para evitar errores de geometría y acelerar la ejecución del código.

## Celdas

Una vez que uno definió una region, es necesario definir una celda y asignarle materiales a esa region utilizando la función `.Cell()`:

In [0]:
umet = openmc.Material()
umet.add_nuclide("U235", 1.0, "ao")
umet.set_density("g/cm3", 19.0)
cel_nucleo_del_reactor = openmc.Cell(fill=umet, region=hemisferio_sur)

En este ejemplo el parámetro `fill=` utiliza un material, pero como veremos una celda también puede llenarse con otras celdas, reunidas en universos.

## Condiciones de contorno

A diferencia de otros códigos Monte Carlo (como MCNP), en los que las condiciones de contorno se establecen en forma volumétrica, las condicionces de contorno en OpenMC se establecen sobre superficies con el parámetro `boundary_type=`. El valor por default para este parámetro es `transmission`, que permite que las partículas atraviesen la superficie sin ningún impedimento. Si esta parámetro se coloca en `vacuum`, OpenMC terminará la historia de las partículas que atraviesan la superficie, generando una condición de contorno de corriente entrante nula (esto es, vacío). Si se coloca el parámetro `reflective` la superficie reflejará las partículas que lleguen a ella, generando una condición de contorno reflejada.

In [0]:
esfera = openmc.Sphere(R=10, boundary_type='vacuum')

Como estas condiciones de contorno pueden aplicarse a cualquier superficie, incluyendo a superficies interiores, hay que tener cuidado de aplicar condiciones de contorno que tengan sentido físico.

## Universos

En OpenMC se denomina *universo* a una colección de celdas con un nombre asignado. Como mínimo debe haber un universo en el sistema, que se denomina `root universe`. Dentro de las celdas de este universo pueden haber otros universos, lo que permite generar estructuras repetidas (por ejemplo, conjuntos de placas combustibles en un reactor tipo MTR). En el siguiente ejemplo definiremos un cubo dentro de una esfera y en base a eso tres celdas: 

1) el interior del cubo, 

2) el exterior del cubo dentro de la esfera, y

3) el exterior de la esfera.

Al interior del cubo lo llenaremos de polietileno y a las esfera de agua. El exterior de la esfera será vacío.

In [0]:
agua = openmc.Material()
agua.add_nuclide("H1", 2.0, 'ao')
agua.add_nuclide("O16", 1.0, 'ao')
agua.set_density('g/cm3', 1.0)

ple = openmc.Material()
ple.add_nuclide("H1", 2.0, 'ao')
ple.add_nuclide("C0", 1.0, 'ao')
ple.set_density('g/cm3', 0.9)

sup_plano1 = openmc.XPlane(x0=-7.5)
sup_plano2 = openmc.XPlane(x0=+7.5)
sup_plano3 = openmc.YPlane(y0=-7.5)
sup_plano4 = openmc.YPlane(y0=+7.5)
sup_plano5 = openmc.ZPlane(z0=-7.5)
sup_plano6 = openmc.ZPlane(z0=+7.5)

sup_esfera = openmc.Sphere(R=15)
adentro_cubo = (+sup_plano1 & -sup_plano2) & \
               (+sup_plano3 & -sup_plano4) & \
               (+sup_plano5 & -sup_plano6) 
afuera_cubo = ~adentro_cubo
adentro_esfera = -sup_esfera
afuera_esfera = +sup_esfera

cel_cubo = openmc.Cell(region=adentro_cubo, fill=ple)
cel_esfera = openmc.Cell(region=adentro_esfera & afuera_cubo, fill=agua)
cel_afuera = openmc.Cell(region=afuera_esfera & afuera_cubo, fill=None)

universo = openmc.Universe(cells= [cel_cubo, cel_esfera, cel_afuera])

Los objetos de clase `Universe` pueden graficarse utilizando el método `.plot()`:

In [0]:
universo.plot(width=(40,40))

Aquí el parámetro `width=` toma una tupla que representa las dimensiones de la región a graficar en cm.